In [8]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import random
import pandas as pd

# Cài đặt cấu hình, tùy chọn khi Selenium mở Google Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

webdriver_service = Service(ChromeDriverManager().install())

# Dùng Chrome
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

# Tạo file CSV và ghi tiêu đề cột
with open('car_Id.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Tên xe', 'Tên hãng xe', 'Năm sản xuất', 'Nơi bán', 'Giá', 'Link sản phẩm'])
limit = 1000

def convert_price(price_str): 
    parts = price_str.split()
    price_in_mil = 0
    for i in range(0, len(parts), 2):
        if parts[i+1] == 'Tỷ':
            price_in_mil += int(parts[i]) * 1000
        elif parts[i+1] == 'Triệu':
            price_in_mil += int(parts[i])
        elif parts[i+1] == 'Ngàn':
            price_in_mil += int(parts[i]) / 1000  # Chuyển 'Ngàn' thành 'Triệu'
    return price_in_mil

for ps in range(limit):
    driver.get(f"https://bonbanh.com/oto/page,{ps}")
    time.sleep(random.randint(3, 5))

    # Tìm tất cả các phần tử có class là "car-item row1" hoặc "car-item row2"
    elems = driver.find_elements(By.CSS_SELECTOR, ".car-item.row1 a[href], .car-item.row2 a[href]")
    link_href=[]
    # Lặp qua các phần tử để lấy thông tin về xe và ghi vào file CSV
    for elem in elems:
        # lấy ra link từng sản phẩm của trang
        link_href.append(elem.get_attribute("href"))
            
        # Lấy thông tin về tên xe
        time.sleep(random.randint(5, 8))
        ten_xe = elem.find_element(By.CSS_SELECTOR, "h3[itemprop='name']").text.strip()
        
        # Lấy thông tin hãng xe
        ten_hang = ten_xe.split()[0]
        
        # Lấy thông tin nơi bán
        try:
            area = elem.find_element(By.CSS_SELECTOR, ".cb4").text.strip()
        except NoSuchElementException:
            area = "N/A"
         # Lấy thông tin về giá xe
        try:
            gia_xe_element = elem.find_element(By.XPATH, ".//div[@class='cb3']//b[@itemprop='price']")
            gia_xe = convert_price(gia_xe_element.text.strip())
        except NoSuchElementException:
            gia_xe = "N/A"
        
        # Lấy thông tin về năm sản xuất
        try:
            nam_san_xuat = int(elem.find_element(By.CLASS_NAME, "cb1").find_element(By.TAG_NAME, "b").text)
        except ValueError:
            nam_san_xuat = None  
     
        # Ghi thông tin vào file CSV
        with open('car_Id.csv', 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([ten_xe, ten_hang, nam_san_xuat, gia_xe, area, link_href[-1]])  # Lấy phần tử cuối cùng của link_href
            
driver.quit()

In [ ]:
# Đọc dữ liệu từ file CSV và tạo DataFrame
df = pd.read_csv('car_Id.csv')

# In ra DataFrame để kiểm tra
print(df)



     Tình trạng xe                                 Tên xe Tên hãng xe  \
0            Xe cũ                  Mazda 3 1.6 AT - 2011       Mazda   
1            Xe cũ             Hyundai Kona 2.0 AT - 2021     Hyundai   
2           Xe mới  Hyundai Accent 1.4 AT Đặc Biệt - 2024     Hyundai   
3            Xe cũ     Kia Carnival Signature 2.2D - 2022         Kia   
4            Xe cũ        Hyundai i10 Grand 1.2 MT - 2015     Hyundai   
...            ...                                    ...         ...   
2146         Xe cũ    Mercedes Benz GLC 300 4Matic - 2019    Mercedes   
2147         Xe cũ               Porsche Macan 2.0 - 2021     Porsche   
2148         Xe cũ     Toyota Fortuner 2.7V 4x2 AT - 2018      Toyota   
2149        Xe mới                 VinFast VF7 Eco - 2024     VinFast   
2150         Xe cũ        Hyundai i10 Grand 1.0 MT - 2014     Hyundai   

      Năm sản xuất   Giá                                      Link sản phẩm  
0             2011   315  https://bonbanh.com